##Projet de Mastercamp

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
valeur = pd.read_csv(r"valeursfoncieres-2023.txt", delimiter="|")

C:\Users\Quentin Damois\AppData\Local\Temp\ipykernel_29080\3540214829.py:1: DtypeWarning: Columns (14,18,23,24,26,28,29,31,32,33,41) have mixed types. Specify dtype option on import or set low_memory=False.
  valeur = pd.read_csv(r"valeursfoncieres-2023.txt", delimiter="|")


On modifie les paramètres de pandas afin de pouvoir voir toutes les colonnes.

In [3]:
# Configurer pandas pour afficher toutes les colonnes
pd.set_option('display.max_columns', None)

# Afficher les 5 premières lignes pour vérifier
print(valeur.head(10))

   Identifiant de document  Reference document  1 Articles CGI  \
0                      NaN                 NaN             NaN   
1                      NaN                 NaN             NaN   
2                      NaN                 NaN             NaN   
3                      NaN                 NaN             NaN   
4                      NaN                 NaN             NaN   
5                      NaN                 NaN             NaN   
6                      NaN                 NaN             NaN   
7                      NaN                 NaN             NaN   
8                      NaN                 NaN             NaN   
9                      NaN                 NaN             NaN   

   2 Articles CGI  3 Articles CGI  4 Articles CGI  5 Articles CGI  \
0             NaN             NaN             NaN             NaN   
1             NaN             NaN             NaN             NaN   
2             NaN             NaN             NaN             NaN 

In [4]:
print(f"Voici toutes les colonnes du dataframe :\n{valeur.info()}")



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3727005 entries, 0 to 3727004
Data columns (total 43 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   Identifiant de document     float64
 1   Reference document          float64
 2   1 Articles CGI              float64
 3   2 Articles CGI              float64
 4   3 Articles CGI              float64
 5   4 Articles CGI              float64
 6   5 Articles CGI              float64
 7   No disposition              int64  
 8   Date mutation               object 
 9   Nature mutation             object 
 10  Valeur fonciere             object 
 11  No voie                     float64
 12  B/T/Q                       object 
 13  Type de voie                object 
 14  Code voie                   object 
 15  Voie                        object 
 16  Code postal                 float64
 17  Commune                     object 
 18  Code departement            object 
 19  Code commune         

In [5]:
valeur_in_cleaning = (valeur.copy())
valeur_in_cleaning.dropna(axis=1, how="all", inplace=True)
print(f"Après avoir enlevée les colonnes vides, il reste les colonnes suivantes :\n{valeur_in_cleaning.info()}")



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3727005 entries, 0 to 3727004
Data columns (total 35 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   No disposition              int64  
 1   Date mutation               object 
 2   Nature mutation             object 
 3   Valeur fonciere             object 
 4   No voie                     float64
 5   B/T/Q                       object 
 6   Type de voie                object 
 7   Code voie                   object 
 8   Voie                        object 
 9   Code postal                 float64
 10  Commune                     object 
 11  Code departement            object 
 12  Code commune                int64  
 13  Prefixe de section          float64
 14  Section                     object 
 15  No plan                     int64  
 16  No Volume                   object 
 17  1er lot                     object 
 18  Surface Carrez du 1er lot   object 
 19  2eme lot             

In [6]:
index_restant = list(valeur_in_cleaning.columns)

print(f"Il reste les colonnes suivantes :")
for i in index_restant:
    print(i)

Il reste les colonnes suivantes :
No disposition
Date mutation
Nature mutation
Valeur fonciere
No voie
B/T/Q
Type de voie
Code voie
Voie
Code postal
Commune
Code departement
Code commune
Prefixe de section
Section
No plan
No Volume
1er lot
Surface Carrez du 1er lot
2eme lot
Surface Carrez du 2eme lot
3eme lot
Surface Carrez du 3eme lot
4eme lot
Surface Carrez du 4eme lot
5eme lot
Surface Carrez du 5eme lot
Nombre de lots
Code type local
Type local
Surface reelle bati
Nombre pieces principales
Nature culture
Nature culture speciale
Surface terrain


On regarde maintenenant les colonnes qui qui possèdes des valeur et que l'on veut garder
Pour cela, on va regarder les colonnes restanes et voir le pour centage de ligne remplis
On récupère tout les colonnes dont moint de 10 % des lignes sont remplis. On va aussi regarder si une colonne comprenant la somme des entrées dans les colonnes sur las surface carrés des lots.

In [7]:



insignificante_column = list()



print("Le pourcentage de ligne remplis pour chaque colonne :")
for i in index_restant:
    percentage_use = len((valeur_in_cleaning[i])[valeur_in_cleaning[i].notna()])/len(valeur_in_cleaning[i])
    print(f"Pour\t {i} :\t{(percentage_use * 100 ):.2f} %")
    if percentage_use < 0.1:
        insignificante_column.append(i)



mask = valeur_in_cleaning["Surface Carrez du 1er lot"].notna() | valeur_in_cleaning["Surface Carrez du 2eme lot"].notna() | valeur_in_cleaning["Surface Carrez du 3eme lot"].notna() | valeur_in_cleaning["Surface Carrez du 4eme lot"].notna() | valeur_in_cleaning["Surface Carrez du 5eme lot"].notna()

nb_nn = len(valeur_in_cleaning[mask]["Surface Carrez du 4eme lot"])
print(f"Pour la somme des surface carrez des lots, le pourcentage de ligne remplis est de {(nb_nn/valeur_in_cleaning.shape[0] * 100):.2f}%")

Le pourcentage de ligne remplis pour chaque colonne :
Pour	 No disposition :	100.00 %
Pour	 Date mutation :	100.00 %
Pour	 Nature mutation :	100.00 %
Pour	 Valeur fonciere :	99.13 %
Pour	 No voie :	62.02 %
Pour	 B/T/Q :	4.42 %
Pour	 Type de voie :	59.58 %
Pour	 Code voie :	99.24 %
Pour	 Voie :	99.23 %
Pour	 Code postal :	99.24 %
Pour	 Commune :	100.00 %
Pour	 Code departement :	100.00 %
Pour	 Code commune :	100.00 %
Pour	 Prefixe de section :	4.62 %
Pour	 Section :	100.00 %
Pour	 No plan :	100.00 %
Pour	 No Volume :	0.22 %
Pour	 1er lot :	30.89 %
Pour	 Surface Carrez du 1er lot :	8.86 %
Pour	 2eme lot :	9.66 %
Pour	 Surface Carrez du 2eme lot :	2.92 %
Pour	 3eme lot :	1.72 %
Pour	 Surface Carrez du 3eme lot :	0.33 %
Pour	 4eme lot :	0.54 %
Pour	 Surface Carrez du 4eme lot :	0.08 %
Pour	 5eme lot :	0.23 %
Pour	 Surface Carrez du 5eme lot :	0.02 %
Pour	 Nombre de lots :	100.00 %
Pour	 Code type local :	57.49 %
Pour	 Type local :	57.49 %
Pour	 Surface reelle bati :	57.42 %
Pour	 Nombre pi

On voit donc q'une colonne contennat la somme des surfac ecarrés de chaque lots serait remplit à 11,9 % donc on vas crée un nouvelle colonnes.
Pour cela on vas d'abord convertir c'est colonne en float :
 - On remplace les NaN par des 0
 - On convertit  les colonnes en entier
 - On fait la somme qui est stocké dans une nouvelle colonne

On remplace les NaN par des 0.

In [8]:

valeur_in_cleaning.fillna(0, inplace=True)



print(f"Maintenant toutes les NaN sont des 0 :\n{valeur_in_cleaning}")

Maintenant toutes les NaN sont des 0 :
         No disposition Date mutation Nature mutation Valeur fonciere  \
0                     1    05/01/2023           Vente      1070000,00   
1                     1    05/01/2023           Vente      1070000,00   
2                     1    05/01/2023           Vente      1070000,00   
3                     1    03/01/2023           Vente       152200,00   
4                     1    05/01/2023           Vente       269000,00   
...                 ...           ...             ...             ...   
3727000               1    05/10/2023    Adjudication      3931000,00   
3727001               1    05/10/2023    Adjudication      3931000,00   
3727002               1    05/10/2023    Adjudication      3931000,00   
3727003               1    28/12/2023           Vente        98250,00   
3727004               1    28/12/2023           Vente        98250,00   

         No voie B/T/Q Type de voie Code voie               Voie  Code postal  \
0  

On enlève convertit les colonne sous forme d'entier

In [9]:
liste_des_lot = ["Surface Carrez du 1er lot", "Surface Carrez du 2eme lot", "Surface Carrez du 3eme lot", "Surface Carrez du 4eme lot", "Surface Carrez du 5eme lot"]

for i in liste_des_lot:
    valeur_in_cleaning[i] = valeur_in_cleaning[i].replace(',', '.', regex=True)
    valeur_in_cleaning[i] = (pd.to_numeric(valeur_in_cleaning[i])).astype("float64")

tempo_string = " ".join(liste_des_lot)
print(f"On peut voir que les colonnes {tempo_string} sont des colonnes de float64 :\n{valeur_in_cleaning[liste_des_lot]}")





On peut voir que les colonnes Surface Carrez du 1er lot Surface Carrez du 2eme lot Surface Carrez du 3eme lot Surface Carrez du 4eme lot Surface Carrez du 5eme lot sont des colonnes de float64 :
         Surface Carrez du 1er lot  Surface Carrez du 2eme lot  \
0                              0.0                         0.0   
1                              0.0                         0.0   
2                              0.0                         0.0   
3                              0.0                         0.0   
4                              0.0                         0.0   
...                            ...                         ...   
3727000                        0.0                         0.0   
3727001                        0.0                         0.0   
3727002                        0.0                         0.0   
3727003                        0.0                         0.0   
3727004                        0.0                         0.0   

         Sur

On crée nouvelle colonne qui contient la somme des surface carrez des lots.

In [10]:

valeur_in_cleaning["Surface Carrez Totale des Lots"] = valeur_in_cleaning[liste_des_lot].sum(axis=1)
tempo_float = len(valeur_in_cleaning[valeur_in_cleaning["Surface Carrez Totale des Lots"] != 0.0]["Surface Carrez Totale des Lots"])/len(valeur_in_cleaning["Surface Carrez Totale des Lots"])

tempo_float *= 100

print(f"On a crée la colonne Surface Carrez Totale des Lots remplis à {tempo_float:.2f} et contenant :")
print(valeur_in_cleaning["Surface Carrez Totale des Lots"])

On a crée la colonne Surface Carrez Totale des Lots remplis à 11.96 et contenant :
0          0.0
1          0.0
2          0.0
3          0.0
4          0.0
          ... 
3727000    0.0
3727001    0.0
3727002    0.0
3727003    0.0
3727004    0.0
Name: Surface Carrez Totale des Lots, Length: 3727005, dtype: float64


On enlève toutes les données dont le pourcentage est inférieur à 10 %.

In [11]:

print("On va enlever : ")
for i in insignificante_column:
    print(f"- {i}")
valeur_in_cleaning.drop(labels=insignificante_column, axis=1, inplace=True)
print(f"Voici les colonnes restantes :")
valeur_in_cleaning.info()


print(valeur_in_cleaning)

On va enlever : 
- B/T/Q
- Prefixe de section
- No Volume
- Surface Carrez du 1er lot
- 2eme lot
- Surface Carrez du 2eme lot
- 3eme lot
- Surface Carrez du 3eme lot
- 4eme lot
- Surface Carrez du 4eme lot
- 5eme lot
- Surface Carrez du 5eme lot
- Nature culture speciale
Voici les colonnes restantes :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3727005 entries, 0 to 3727004
Data columns (total 23 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   No disposition                  int64  
 1   Date mutation                   object 
 2   Nature mutation                 object 
 3   Valeur fonciere                 object 
 4   No voie                         float64
 5   Type de voie                    object 
 6   Code voie                       object 
 7   Voie                            object 
 8   Code postal                     float64
 9   Commune                         object 
 10  Code departement                object 

Sur les lignes restantes, nous allons enlever tous les numéros d'identifications

In [12]:
valeur_in_cleaning.drop(labels=["No disposition", "No voie", "Voie",  "No plan"], axis=1, inplace=True)

print(f"Voici les colonnes restantes :")
valeur_in_cleaning.info()

Voici les colonnes restantes :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3727005 entries, 0 to 3727004
Data columns (total 19 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   Date mutation                   object 
 1   Nature mutation                 object 
 2   Valeur fonciere                 object 
 3   Type de voie                    object 
 4   Code voie                       object 
 5   Code postal                     float64
 6   Commune                         object 
 7   Code departement                object 
 8   Code commune                    int64  
 9   Section                         object 
 10  1er lot                         object 
 11  Nombre de lots                  int64  
 12  Code type local                 float64
 13  Type local                      object 
 14  Surface reelle bati             float64
 15  Nombre pieces principales       float64
 16  Nature culture                  object 
 

On convertit les colonnes restantes en entier

In [16]:
a_convertir=['Valeur fonciere','Code postal','Nombre de lots','Code type local','Surface reelle bati','Nombre pieces principales','Surface terrain','Code commune']

print(valeur_in_cleaning)


for i in a_convertir:
    print(i)
    valeur_in_cleaning[i] = valeur_in_cleaning[i].replace(',', '.', regex=True)
    valeur_in_cleaning[i] = (pd.to_numeric(valeur_in_cleaning[i])).astype(float) # changer le astype(int) si vous volez changer les chifre en int, n'oublier pas de recherger a nouveau tout le document

valeur_in_cleaning.fillna(0)

print(f"Voici les types de colonnes après la convertion :")
valeur_in_cleaning.info()

print(len(valeur_in_cleaning["Valeur fonciere"].isna()))

print(valeur_in_cleaning)

        Date mutation Nature mutation  Valeur fonciere Type de voie Code voie  \
0          05/01/2023           Vente        1070000.0          ALL      0124   
1          05/01/2023           Vente        1070000.0          ALL      0124   
2          05/01/2023           Vente        1070000.0          ALL      0124   
3          03/01/2023           Vente         152200.0          RTE      0107   
4          05/01/2023           Vente         269000.0          CHE      0040   
...               ...             ...              ...          ...       ...   
3727000    05/10/2023    Adjudication        3931000.0           AV    1832.0   
3727001    05/10/2023    Adjudication        3931000.0           AV    1832.0   
3727002    05/10/2023    Adjudication        3931000.0           AV    1832.0   
3727003    28/12/2023           Vente          98250.0          RUE    8892.0   
3727004    28/12/2023           Vente          98250.0          RUE    8892.0   

         Code postal       

On convertit maintenant la colone date mutation de string en date.

In [17]:
function_convert_to_date = lambda x: datetime.strptime(x, "%d/%m/%Y")
valeur_in_cleaning["Date mutation"] = valeur_in_cleaning["Date mutation"].apply(func=function_convert_to_date)

Comme on peut le voir ci-dessus, le type de la colonne Date mutation  est maintenant datetime64[ns].

In [18]:
print(f"Voici les colonnes restantes :")
valeur_in_cleaning["Date mutation"].info()

Voici les colonnes restantes :
<class 'pandas.core.series.Series'>
RangeIndex: 3727005 entries, 0 to 3727004
Series name: Date mutation
Non-Null Count    Dtype         
--------------    -----         
3727005 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 28.4 MB


In [19]:
print(f"On ad ocn le tableau suivant :\n{valeur_in_cleaning}")

On ad ocn le tableau suivant :
        Date mutation Nature mutation  Valeur fonciere Type de voie Code voie  \
0          2023-01-05           Vente        1070000.0          ALL      0124   
1          2023-01-05           Vente        1070000.0          ALL      0124   
2          2023-01-05           Vente        1070000.0          ALL      0124   
3          2023-01-03           Vente         152200.0          RTE      0107   
4          2023-01-05           Vente         269000.0          CHE      0040   
...               ...             ...              ...          ...       ...   
3727000    2023-10-05    Adjudication        3931000.0           AV    1832.0   
3727001    2023-10-05    Adjudication        3931000.0           AV    1832.0   
3727002    2023-10-05    Adjudication        3931000.0           AV    1832.0   
3727003    2023-12-28           Vente          98250.0          RUE    8892.0   
3727004    2023-12-28           Vente          98250.0          RUE    8892.0 

<h1>Détection des doublons</h1>
<h2>premiere partie :</h2> Identifier les doublonsgrâce à un mask (<code>True</code> indique un doublon)

In [20]:
duplicates = valeur_in_cleaning.duplicated()

<h2>deuxieme partie :</h2>
Afficher les doublons

In [21]:
print("Doublons:")
print(valeur_in_cleaning[duplicates])

Doublons:
        Date mutation Nature mutation  Valeur fonciere Type de voie Code voie  \
6          2023-01-05           Vente         269000.0          CHE      0040   
31         2023-01-04           Vente         430000.0          CHE      0083   
39         2023-01-04           Vente         430000.0          CHE      0083   
40         2023-01-04           Vente         430000.0          CHE      0083   
41         2023-01-04           Vente         430000.0          CHE      0083   
...               ...             ...              ...          ...       ...   
3726939    2023-12-15           Vente       47000000.0          RUE    1591.0   
3726940    2023-12-15           Vente       47000000.0          RUE    1591.0   
3726941    2023-12-15           Vente       47000000.0          RUE    1591.0   
3726942    2023-12-15           Vente       47000000.0          RUE    1591.0   
3726943    2023-12-15           Vente       47000000.0          RUE    1591.0   

         Code pos

<h2>3e partie :</h2>
Supprimer les doublons

In [22]:
data_cleaned = valeur_in_cleaning.drop_duplicates()


<h2>4e partie :</h2>
Afficher un aperçu des données après avoir supprimé les doublons

In [23]:
print("Après avoir supprimé les doublons:")
print(data_cleaned.head())

Après avoir supprimé les doublons:
  Date mutation Nature mutation  Valeur fonciere Type de voie Code voie  \
0    2023-01-05           Vente        1070000.0          ALL      0124   
1    2023-01-05           Vente        1070000.0          ALL      0124   
2    2023-01-05           Vente        1070000.0          ALL      0124   
3    2023-01-03           Vente         152200.0          RTE      0107   
4    2023-01-05           Vente         269000.0          CHE      0040   

   Code postal              Commune Code departement  Code commune Section  \
0       1630.0     ST-GENIS-POUILLY                1         354.0      BD   
1       1630.0     ST-GENIS-POUILLY                1         354.0      BD   
2       1630.0     ST-GENIS-POUILLY                1         354.0      BD   
3       1450.0    SERRIERES-SUR-AIN                1         404.0       D   
4       1800.0  SAINT-JEAN-DE-NIOST                1         361.0       B   

  1er lot  Nombre de lots  Code type local   

<h2>5e partie :</h2>
Afficher le nombre de doublons trouvés et supprimés

In [24]:
print(f"Nombre de doublons trouvés et supprimés: {duplicates.sum()}")

Nombre de doublons trouvés et supprimés: 292941


<h1>Aprentissage machine</h1>
On vas donc d'abord calculer replacer les chaines de caractères par des entiers.

In [25]:
valeur_in_cleaning.info()
print(valeur_in_cleaning["Commune"].head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3727005 entries, 0 to 3727004
Data columns (total 19 columns):
 #   Column                          Dtype         
---  ------                          -----         
 0   Date mutation                   datetime64[ns]
 1   Nature mutation                 object        
 2   Valeur fonciere                 float64       
 3   Type de voie                    object        
 4   Code voie                       object        
 5   Code postal                     float64       
 6   Commune                         object        
 7   Code departement                object        
 8   Code commune                    float64       
 9   Section                         object        
 10  1er lot                         object        
 11  Nombre de lots                  float64       
 12  Code type local                 float64       
 13  Type local                      object        
 14  Surface reelle bati             float64       
 15

On doit donc remplacer par des entier Nature mutation, Commune, code departement, Section, Type local, Nature culture.